# Pawpularity Model with DNN on Meta Data

## Table of Contents
- Summary
- Set up
- Import datasets
- Data Preprocessing
- Model Development
- Model Evaluation
- Submission


## Summary
In this notebook, I am going use KeraTuner to find the best Model that fits only on Meta Data. So that I can improve total Performance of Model that use both Image Data and Meta Data. Some of my notebooks that use result can be found in:
* [Pawpularity with EfficientNet: [Training]](https://www.kaggle.com/lonnieqin/pawpularity-with-efficientnet-training)
* [Pawpularity with EfficientNet: [Inference]](https://www.kaggle.com/lonnieqin/pawpularity-with-efficientnet-inference)
* [TensorFlow multi-input Pet Pawpularity Model](https://www.kaggle.com/lonnieqin/tensorflow-multi-input-pet-pawpularity-model)

## Set up

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import keras_tuner as kt

## Import datasets

In [ ]:
train = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
sample_submission = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")

## Data Preprocessing

In [ ]:
train.head()

In [ ]:
train["Pawpularity"].hist()

## Model Development

In [ ]:
batch_size = 128 # Batch Size
train_on_fold = None # Which fold to train, None to train on all folds.
tabular_columns = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']

### RMSE loss function

In [ ]:
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean((y_true -  y_pred) ** 2))

### The Model

In [ ]:
def build_model(hp):
    inputs = tf.keras.layers.Input((len(tabular_columns)))
    width = hp.Choice('width', [8, 16, 32, 64])
    depth = hp.Choice('depth', [3, 6, 9, 12])
    activation = "relu"
    dropout = hp.Choice('dropout', [0.0, 0.1, 0.2])
    use_batch_norm = hp.Choice('use_batch_norm', [True, False])
    # Whether to use MSE or RMSE is worth to have a try
    loss_function = hp.Choice("loss_function", ["mse", "rmse"])
    kernel_regularizer = hp.Choice("kernel_regularizer", ["none", "l1", "l2", "l1_l2"])
    acutal_kernel_regularizer = None
    if kernel_regularizer == "l1":
        acutal_kernel_regularizer = keras.regularizers.l1()
    if kernel_regularizer == "l2":
        acutal_kernel_regularizer = keras.regularizers.l2()
    if kernel_regularizer == "l1_l2":
        acutal_kernel_regularizer = keras.regularizers.l1_l2()
    for i in range(depth):
        if i == 0:
            x = inputs
           
        x = keras.layers.Dense(
            width, 
            activation=activation,
            kernel_regularizer=acutal_kernel_regularizer
        )(x)
        if (i + 1) % 3 == 0:
            if dropout > 0:
                x = keras.layers.Dropout(dropout)(x)
            if use_batch_norm:
                x = keras.layers.BatchNormalization()(x)
            x = keras.layers.Concatenate()([x, inputs])
    output = keras.layers.Dense(1, activation="relu")(x)
    model = keras.Model(inputs=inputs, outputs=output)
    adam = keras.optimizers.Adam(learning_rate=hp.Float("learing_rate", 1e-5, 5e-3))
    loss = "mse" if loss_function == "mse" else rmse
    model.compile(loss=loss, optimizer=adam, metrics=["mae", rmse])
    return model

### Preprocess function

In [ ]:
def preprocess(x, y):
    x = tf.cast(x, tf.float32)
    y = tf.cast(y, tf.float32)
    print(x, y)
    return x, y

### HyperParmeter Tuning

In [ ]:
tf.keras.backend.clear_session()
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
for index, (train_indices, val_indices) in enumerate(kfold.split(train)):
    if train_on_fold is not None and train_on_fold != index:
        continue
    train_features = train.loc[train_indices, tabular_columns]
    train_targets = train.loc[train_indices, ["Pawpularity"]]
    val_features = train.loc[val_indices, tabular_columns]
    val_targets = train.loc[val_indices, ["Pawpularity"]]
    checkpoint_path = "model_%d.h5"%(index)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        checkpoint_path, 
        save_best_only=True
    )
    early_stop = tf.keras.callbacks.EarlyStopping(
        min_delta=1e-4, 
        patience=10
    )
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        factor=0.3,
        patience=2, 
        min_lr=1e-7
    )
    callbacks = [early_stop, checkpoint, reduce_lr]
    print(train_features.shape, train_targets.shape)
    print(val_features.shape, val_targets.shape)
    train_ds = tf.data.Dataset.from_tensor_slices((train_features, train_targets)).map(preprocess).shuffle(512).batch(batch_size).cache().prefetch(2)
    val_ds = tf.data.Dataset.from_tensor_slices((val_features, val_targets)).map(preprocess).batch(batch_size).cache().prefetch(2)
    tuner = kt.RandomSearch(
        build_model,
        objective=kt.Objective("val_rmse", direction="min"),
        max_trials=100,
        directory="directory"
    )
    tuner.search(train_ds, validation_data=val_ds, epochs=10, callbacks=callbacks)
    break

### Best Architecture

In [ ]:
best_model = tuner.get_best_models()[0]
keras.utils.plot_model(best_model, show_shapes=True)

### Best parameters

In [ ]:
"""
 {'width': 64,
 'depth': 6,
 'dropout': 0.1,
 'use_batch_norm': 0,
 'loss_function': 'mse',
 'kernel_regularizer': 'none',
 'learing_rate': 0.0038644865099609653}
"""
best_hp = tuner.get_best_hyperparameters()[0]
best_hp.get_config()["values"]

### Keep Trainnig with best Parameters

In [ ]:
models = []
for index, (train_indices, val_indices) in enumerate(kfold.split(train)):
    if train_on_fold is not None and train_on_fold != index:
        continue
    train_features = train.loc[train_indices, tabular_columns]
    train_targets = train.loc[train_indices, ["Pawpularity"]]
    val_features = train.loc[val_indices, tabular_columns]
    val_targets = train.loc[val_indices, ["Pawpularity"]]
    checkpoint_path = "model_%d.h5"%(index)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        checkpoint_path, 
        save_best_only=True
    )
    early_stop = tf.keras.callbacks.EarlyStopping(
        min_delta=1e-4, 
        patience=10
    )
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        factor=0.3,
        patience=2, 
        min_lr=1e-7
    )
    callbacks = [early_stop, checkpoint, reduce_lr]
    print(train_features.shape, train_targets.shape)
    print(val_features.shape, val_targets.shape)
    train_ds = tf.data.Dataset.from_tensor_slices((train_features, train_targets)).map(preprocess).shuffle(512).batch(batch_size).cache().prefetch(2)
    val_ds = tf.data.Dataset.from_tensor_slices((val_features, val_targets)).map(preprocess).batch(batch_size).cache().prefetch(2)
    model = tuner.hypermodel.build(best_hp)
    history = model.fit(train_ds, epochs=50, validation_data=val_ds, callbacks=callbacks)
    model.load_weights(checkpoint_path)
    models.append(model)

## Submission

In [ ]:
def preprocess_test_data(x):
    x = tf.cast(x, tf.float32)
    print(x)
    return x

In [ ]:
test_ds = tf.data.Dataset.from_tensor_slices((test[tabular_columns])).map(preprocess_test_data).batch(batch_size).prefetch(2)

In [ ]:
total_results = []
for model in models:
    total_results.append(model.predict(test_ds).reshape(-1))
results = np.mean(total_results, axis=0).reshape(-1)
sample_submission["Pawpularity"] = results
sample_submission.to_csv("submission.csv", index=False)